In [1]:
from storygrabber.modules.storygraph import Storygraph
from storygrabber.modules.lazylibrarian import LazyLibrarian
from storygrabber.modules.audiobookshelf import AudioBookShelf
import dotenv

dotenv.load_dotenv()

LL_HOST = "192.168.1.222"
LL_PORT = "5299"
# LazyLibrarian API key (required to enable integration)
LL_API_KEY = "0a3431081e97cb4ed057d66bff4a5e79"
# Use HTTPS (true/false)
LL_HTTPS = False
# In-memory cache TTL for LazyLibrarian getAllBooks (seconds)
LL_CACHE_TTL = "60"


In [2]:
sg = Storygraph()
import asyncio
from storygrabber.modules.util import read_cache

abs = AudioBookShelf()
cached = read_cache("storygraph", "0xchloe") or {}
sg_books = cached.get("books")
ll_books = LazyLibrarian(LL_HOST, int(LL_PORT), LL_API_KEY, LL_HTTPS).get_all_books()
abs_books = []  # AudiobookShelf integration not yet implemented
abs_libraries = await abs.get_libraries()
for library in abs_libraries:
    abs_books.extend(await abs.get_library_items(library.id))
lib_map = {lib.id: lib.name for lib in abs_libraries}

http://192.168.1.222:8191/v1


2026-01-27 00:34:05.020 | DEBUG    | storygrabber.modules.lazylibrarian:get_all_books:95 - Getting all books from database


{"status": "ok", "message": "Session created successfully.", "session": "e1d4a292-fb17-11f0-8e8c-ceaa83b457d0", "startTimestamp": 1769474044010, "endTimestamp": 1769474044857, "version": "3.3.24"}


2026-01-27 00:34:10.622 | DEBUG    | storygrabber.modules.lazylibrarian:_make_request:40 - Raw response text (first 200 chars): [{"AuthorID": "OL9155324A", "AuthorName": "Samantha Kolesnik", "AuthorLink": "https://www.goodreads.com/author/show/19735299.Samantha_Kolesnik", "BookName": "Waif", "BookSub": "", "BookGenre": "Unknow
2026-01-27 00:34:11.052 | DEBUG    | storygrabber.modules.lazylibrarian:_make_request:48 - Successfully parsed JSON response, type: <class 'list'>
2026-01-27 00:34:11.053 | DEBUG    | storygrabber.modules.lazylibrarian:get_all_books:97 - get_all_books response count: 60409
2026-01-27 00:34:11.055 | DEBUG    | storygrabber.modules.lazylibrarian:_normalize_response:71 - Normalizing response of type: <class 'list'>
2026-01-27 00:34:11.056 | DEBUG    | storygrabber.modules.lazylibrarian:_normalize_response:77 - Response is a list with 60409 items - normalizing to dict
2026-01-27 00:34:11.230 | ERROR    | storygrabber.modules.audiobookshelf:get_libraries:236 - [{'id':

In [8]:
import pandas as pd
from typing import Optional, Any

from storygrabber.modules.util import abs_items_aggregated_df


def records_to_df(
    records: list | dict[str, Any], link_base: Optional[str] = None
) -> pd.DataFrame:
    """
    Convert a list of book dicts (or a LazyLibrarian-style dict wrapper) into a normalized DataFrame.

    - Accepts either a list of dicts or a dict like {"success": True, "data": [...]}
    - Renames keys to snake_case column names
    - Keeps fields as strings unless strict parsing requested
    - Optionally prepends link_base to BookLink when it starts with '/'
    """
    # LazyLibrarian returns a normalized dict sometimes: {"success": True, "data": [...]}
    if isinstance(records, dict):
        if isinstance(records.get("data"), list):
            records = records["data"]
        else:
            # Try to find the first list value in the dict
            for v in records.values():
                if isinstance(v, list):
                    records = v
                    break

    if not records:
        return pd.DataFrame()

    # Normalize JSON -> flat DataFrame
    df = pd.json_normalize(records)

    # Column rename map
    rename_map = {
        "AuthorID": "author_id",
        "AuthorName": "author",
        "AuthorLink": "author_link",
        "BookName": "title",
        "BookSub": "subtitle",
        "BookGenre": "genre",
        "BookIsbn": "isbn",
        "BookPub": "publisher",
        "BookRate": "rating",
        "BookImg": "image",
        "BookPages": "pages",
        "BookLink": "link",
        "BookID": "book_id",
        "BookDate": "published_year",
        "BookLang": "language",
        "BookAdded": "added",
        "Status": "status",
        "AudioStatus": "audio_status",
        "BookLibrary": "book_library",
        "AudioLibrary": "audio_library",
    }
    df = df.rename(columns=rename_map)

    # Keep fields as strings; only normalize missing values to None and strip whitespace
    for _col in ("rating", "pages", "added", "published_year"):
        if _col in df.columns:
            df[_col] = df[_col].apply(lambda x: None if pd.isna(x) else str(x).strip())

    # Optionally prepend base to relative links (e.g. "/works/...")
    if link_base and "link" in df.columns:
        df["link"] = df["link"].apply(
            lambda x: f"{link_base}{x}"
            if isinstance(x, str) and x.startswith("/")
            else x
        )

    # Pick / reorder relevant columns (keep only existing)
    preferred_order = [
        "book_id",
        "title",
        "subtitle",
        "author_id",
        "author",
        "author_name",
        "author_link",
        "isbn",
        "publisher",
        "rating",
        "pages",
        "published_year",
        "language",
        "added",
        "status",
        "audio_status",
        "book_library",
        "audio_library",
        "genre",
        "image",
        "link",
    ]
    cols = [c for c in preferred_order if c in df.columns]
    return df[cols]


def _strip_subtitles(title: str) -> str:
    """
    Remove subtitles from a book title string.

    E.g. "The Great Book: A Novel" -> "The Great Book"
    """
    if not isinstance(title, str):
        return title
    # Split on common subtitle separators
    for sep in [":", "-", "–", "|"]:
        if sep in title:
            return title.split(sep)[0].strip()
    return title.strip()


def storygraph_records_to_df(
    records: list | dict, link_base: Optional[str] = None
) -> pd.DataFrame:
    """
    Convert Storygraph `get_books` output into a normalized DataFrame.

    Accepts records like: [(link, title, author), ...] or a dict wrapper and returns
    columns: `id`, `link`, `title`, `author` (all strings).
    """
    # Unwrap dict wrappers (e.g. {"data": [...]})
    if isinstance(records, dict):
        if isinstance(records.get("data"), list):
            records = records["data"]
        else:
            for v in records.values():
                if isinstance(v, list):
                    records = v
                    break

    if not records:
        return pd.DataFrame()

    from urllib.parse import urlparse

    rows = []
    for item in records:
        if not item:
            continue
        # tuple/list format: (link, title, author)
        if isinstance(item, (list, tuple)):
            link = str(item[0]) if len(item) > 0 else None
            title = str(item[1]) if len(item) > 1 else None
            author = str(item[2]) if len(item) > 2 else None
        # dicts with keys
        elif isinstance(item, dict):
            link = (
                item.get("link")
                or item.get("url")
                or item.get("book_url")
                or item.get("BookLink")
            )
            title = item.get("title") or item.get("BookName")
            author = item.get("author") or item.get("AuthorName")
        else:
            # fallback to string
            s = str(item)
            link = None
            title = s
            author = None

        # derive id from final path segment of link when possible
        id_ = None
        if link:
            path = urlparse(link).path.rstrip("/")
            id_ = path.split("/")[-1] if path else None
            if isinstance(link, str) and link.startswith("/") and link_base:
                link = f"{link_base}{link}"

        rows.append(
            {
                "id": id_,
                "link": link,
                "title": title.strip() if isinstance(title, str) else title,
                "author": author.strip() if isinstance(author, str) else author,
            }
        )

    df_sg = pd.DataFrame(rows, columns=["id", "link", "title", "author"])
    df_sg["title"] = df_sg["title"].apply(lambda x: _strip_subtitles(x))

    # Keep values as strings
    for c in df_sg.columns:
        df_sg[c] = df_sg[c].apply(lambda x: None if pd.isna(x) else str(x).strip())

    return df_sg


sg_df = storygraph_records_to_df(sg_books)
ll_df = records_to_df(ll_books)
abs_client = AudioBookShelf()
abs_libraries = await abs_client.get_libraries()
lib_map = {lib.id: lib.name for lib in abs_libraries}
abs_df = abs_items_aggregated_df(abs_books)
merged_df = sg_df.merge(
    ll_df,
    on=["title", "author"],
    how="left",
    suffixes=("_sg", "_ll"),
)
# Merge aggregated ABS fields (adds abs_in_ebook/abs_in_audiobook and library lists)
abs_merge = merged_df.merge(
    abs_df,
    on=["title", "author"],
    how="left",
)

import json
json.dumps(abs_merge.to_dict(orient="records")), 200




2026-01-27 04:22:51.281 | ERROR    | storygrabber.modules.audiobookshelf:get_libraries:236 - [{'id': '9e0f554f-3a38-498d-9ae3-991358e8e076', 'name': 'Audiobooks', 'folders': [{'id': 'd6fa3194-90d1-45e1-ad92-e95f9833d9af', 'fullPath': '/audiobooks', 'libraryId': '9e0f554f-3a38-498d-9ae3-991358e8e076', 'addedAt': 1693527864780}], 'displayOrder': 1, 'icon': 'audiobookshelf', 'mediaType': 'book', 'provider': 'audible', 'settings': {'coverAspectRatio': 1, 'disableWatcher': False, 'skipMatchingMediaWithAsin': False, 'skipMatchingMediaWithIsbn': False, 'autoScanCronExpression': '0 * * * *', 'audiobooksOnly': True, 'epubsAllowScriptedContent': False, 'hideSingleBookSeries': False, 'onlyShowLaterBooksInContinueSeries': False, 'metadataPrecedence': ['folderStructure', 'audioMetatags', 'txtFiles', 'opfFile', 'absMetadata'], 'podcastSearchRegion': 'us'}, 'lastScan': 1769486412133, 'lastScanVersion': '2.28.0', 'createdAt': 1693527864764, 'lastUpdate': 1769486412133}, {'id': 'a6090e66-d0cc-4cfa-9739

('[{"id": "ff94854a-ad06-45c4-8de3-acc7050364ba", "link_sg": "https://app.thestorygraph.com/books/ff94854a-ad06-45c4-8de3-acc7050364ba", "title": "Under the Eye of Power", "author": "Colin Dickey", "book_id": "63024270", "subtitle": "How Fear of Secret Societies Shapes American Democracy", "author_id": "2943194", "author_link": "https://www.goodreads.com/author/show/2943194.Colin_Dickey", "isbn": "", "publisher": "Penguin", "rating": "3.9", "pages": "369.0", "published_year": "2023-07-11", "language": "eng", "added": "2025-03-27T00:00:00Z", "status": "Open", "audio_status": "Open", "book_library": "2026-01-21T10:22:32Z", "audio_library": "2025-06-23T05:34:58Z", "genre": "non-fiction, history, politics, audiobooks", "image": "cache/book/63024270.jpg", "link_ll": "https://www.goodreads.com/book/show/63024270-under-the-eye-of-power", "abs_in_ebook": true, "abs_in_audiobook": true, "abs_ebook_libraries": "5c95b8ac-59d5-4440-ab29-e09cf54eb709", "abs_audiobook_libraries": "9e0f554f-3a38-498d